# Understanding Agents vs Prompt engineering

Link to chatGPT discussion: https://chatgpt.com/share/683650cc-0450-8001-a54e-0755978d8b1c

In [9]:
# displays the conda environment used by the current process
!conda info --envs | grep '*' | awk '{print $1}'

agents


### Is the name Agents in GAI the same as writing a request to LLMs using prompt engineering  
### and parameters as temperature, top k, role, user and system?

Great question. The term **"Agents"** in the context of **Generative AI (GAI)**—especially as used by OpenAI and similar frameworks—is related to, but not the same as, simply sending a request to an LLM with a well-crafted prompt and parameters like temperature, top-k, role, etc.

### Here's the key difference:

#### 1. **Prompting an LLM**

This involves:

* A single input (or a chat history) with roles (`system`, `user`, `assistant`)
* Prompt engineering (writing effective instructions)
* Parameters like `temperature`, `top_p`, `max_tokens`, etc.
* No persistent memory or external tool usage (unless manually coded)

This is **stateless**: each request is independent unless you manually pass conversation history.

#### 2. **Agents in GAI (like OpenAI’s GPTs or LangChain Agents)**

This involves:

* An **LLM + memory + tools + planning + acting**
* Capable of **autonomous decision-making**, like:

  * Figuring out sub-tasks
  * Calling tools/APIs/web searches
  * Keeping track of state or objectives
* Can use **multi-turn reasoning** and **external actions** (like searching the web or running Python code)
* Often built on **frameworks** like LangChain, OpenAI's `function calling`, ReAct, or OpenAI's GPT-based "Agents" system

This is **stateful** and **autonomous**: it can manage tasks that involve planning, tool use, and remembering context over time.

---

### Analogy

* **Prompting an LLM** is like giving a very smart assistant a one-time instruction.
* **Using an Agent** is like hiring a capable intern who can:

  * Understand goals
  * Ask clarifying questions
  * Use tools like calculators, databases, or browsers
  * Keep track of progress over time
  * Make decisions along the way

---

### Conclusion

**No, "Agents" is not the same as just prompt engineering with parameters**—it's a more advanced construct that combines LLMs with structured decision-making, memory, and sometimes tool use.

If you're doing simple prompting with temperature/top-k/roles, you're not yet using Agents—but you can *build* an Agent by combining those capabilities in a structured way.


## Prompting Example (Simple)

In [ ]:
#%pip install --quiet openai

import openai
import os

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that explains Python code."},
        {"role": "user", "content": "Explain what a Python list comprehension is."}
    ],
    temperature=0.7
)

print(response.choices[0].message.content)


Note: you may need to restart the kernel to use updated packages.
A Python list comprehension is a compact way of creating a Python list. It's a syntactic construct that enables lists to be created from other lists (or any iterable) by applying an expression to each element in the original list.

The basic syntax of a list comprehension is:
```python
[expression for element in iterable]
```
Here, `expression` is a Python expression that is applied to each `element` in the `iterable`.

Here's an example of a list comprehension that creates a new list containing the squares of all numbers in an existing list:

```python
numbers = [1, 2, 3, 4, 5]
squares = [n**2 for n in numbers]
```
After running this code, `squares` would be `[1, 4, 9, 16, 25]`.

You can also add an optional `if` condition to a list comprehension to filter the elements from the original list that you want to include in the new list. Here's an example:

```python
numbers = [1, 2, 3, 4, 5]
even_squares = [n**2 for n in nu

## Agent Example (LangChain with Tool Use)

In [4]:
#%pip install numexpr

from langchain.agents import initialize_agent, load_tools
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
import os

#os.environ["OPENAI_API_KEY"] = "your-api-key"
api_key=os.getenv("OPENAI_API_KEY")
# Initialize the LLM
llm = OpenAI(temperature=0)

# Load tools, like a calculator
tools = load_tools(["llm-math"], llm=llm)

# Create a reasoning agent that can use tools
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Ask a question that requires tool use
agent.run("What is the square root of 8461 divided by 3?")


/tmp/ipykernel_63798/3105818160.py:17: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipykernel_63798/3105818160.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is the square root of 8461 divided by 3?")




> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem.
Action: Calculator
Action Input: square root of 8461 divided by 3
Observation: Answer: 30.661231402393337
Thought: I now know the final answer.
Final Answer: 30.661231402393337

> Finished chain.


'30.661231402393337'

## Agent with Memory and Tools (LangChain)
Let's build on the agent example by adding memory and tool use, using LangChain. This agent will:

Remember previous interactions

Use tools (like a calculator or a text search tool)

Work over multiple turns

In [2]:
import os
from langchain.agents import initialize_agent, load_tools
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

# 🔐 Set your OpenAI API key
api_key=os.getenv("OPENAI_API_KEY")
#print(f"Using OpenAI API key: {api_key}")

# Initialize LLM
llm = OpenAI(temperature=0)

# Load tools (e.g., calculator)
tools = load_tools(["llm-math"], llm=llm)

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Create an agent with memory and tool use
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# 🗨️ Multi-turn interaction
agent.run("Hi, I’m working on a project involving Fibonacci numbers.")

agent.run("What is the 20th Fibonacci number?")

agent.run("Can you remind me what I said earlier?")




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: What is the 10th Fibonacci number?
Observation: Answer: 55.00363612324741
Thought: Do I need to use a tool? No
AI: That's interesting! Fibonacci numbers are a fascinating mathematical concept. Did you know that they were first introduced by Leonardo Fibonacci in the 12th century? They have many interesting properties and applications in fields such as computer science and finance. Is there anything specific you would like to know about Fibonacci numbers?

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 20th Fibonacci number
Observation: Answer: 6765.000029563931
Thought: Do I need to use a tool? No
AI: The 20th Fibonacci number is 6765.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Sure, you mentioned that you were working on a project involvi

'Sure, you mentioned that you were working on a project involving Fibonacci numbers. Is there anything else you would like to know about them?'

## Example 1: File Upload + Summarization Agent
This agent will:

Read a local file (e.g. a .txt or .pdf)

Summarize it using an LLM

In [2]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Set your OpenAI key
api_key=os.getenv("OPENAI_API_KEY")

# Step 1: Load a PDF file
loader = PyPDFLoader("./paper.pdf")
documents = loader.load()

# Step 2: Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

# Step 3: Create vector index
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

# Step 4: Build a QA system with retrieval
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=retriever)

# Step 5: Ask questions or summarize
response = qa_chain.run("Summarize this document for me.")
print(response)


/tmp/ipykernel_69343/2018193396.py:26: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=retriever)
/tmp/ipykernel_69343/2018193396.py:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run("Summarize this document for me.")



The document discusses the development of a methodology that can use unlabeled data to extract dense vector representations of time series through input denoising. It also explores the impact of additional labels on performance and the potential use of different masking patterns for different objectives. Future work includes a quantitative comparison with existing methods.


## Example 2: Web Search Agent + Memory
This agent:

Has memory

Can access the internet (via Tavily tool)

Remembers what you’ve asked across turns



In [ ]:
import os
from langchain.agents import initialize_agent, load_tools
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

api_key=os.getenv("OPENAI_API_KEY")
tavily_key=os.getenv("TAVILY_API_KEY")



llm = OpenAI(temperature=0)
tools = load_tools(["tavily-search"], llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Ask questions across turns
agent.run("What is the latest news about quantum computing?")
agent.run("Summarize that for a high school student.")
agent.run("Remind me what I just asked you.")
